In [ ]:
import io
import torch
import torch.nn as nn
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from torchvision.utils import make_grid 
from torchvision.models import resnet50, ResNet50_Weights
import lightning as L
import torch.nn.functional as F
from torchmetrics.classification import BinaryF1Score, BinaryConfusionMatrix, BinaryAccuracy, BinaryRecall, BinaryPrecision
from torchmetrics import MetricCollection
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
import PIL.Image
from torchvision.transforms import ToTensor
import numpy as np
import seaborn as sns

## Hyperparameters

In [ ]:
BATCH_SIZE = 32
SHUFFLE = False # Warning message that it should be turned off for train/val loaders check later
NUM_WORERS = 7 # 7 because that what it suggested in a warning message
PERSISTENT_WORKERS = True # Suggested to do this in a warning message for fatser init

CUSTOM_TRAIN_VAL_SPLIT = True
TRAIN_FRACTION = 0.8
VALIDATION_FRACTION = 1 - TRAIN_FRACTION

LEARNING_RATE = 1e-3
EPOCHS = 100

## Load data

In [ ]:
CLASS_LABELS = ["Normal", "Pneumonia"]
DATA_DIR = "chest_xray" # Change this to chest_xray folder
data_dir_train = DATA_DIR + "/train"
data_dir_val = DATA_DIR + "/val"
data_dir_test = DATA_DIR + "/test"

# Some desired transforms for ResNet50
# https://pytorch.org/vision/stable/models/generated/torchvision.models.resnet50.html#torchvision.models.resnet50
transform = transforms.Compose(
    [
        transforms.Resize(size = (256, 256)),
        transforms.CenterCrop(size=(224, 224)),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)) # 
    ]
)
dataset_train = datasets.ImageFolder(data_dir_train, transform)
dataset_val = datasets.ImageFolder(data_dir_val, transform)

if CUSTOM_TRAIN_VAL_SPLIT:
    dataset = torch.utils.data.ConcatDataset([dataset_train, dataset_val])
    dataset_train, dataset_val = torch.utils.data.random_split(dataset, [TRAIN_FRACTION, VALIDATION_FRACTION])

dataset_test = datasets.ImageFolder(data_dir_test, transform)

train_loader = DataLoader(
    dataset_train, 
    batch_size=BATCH_SIZE, 
    shuffle=SHUFFLE, 
    num_workers=NUM_WORERS, 
    persistent_workers= PERSISTENT_WORKERS
)
val_loader = DataLoader(
    dataset_val, 
    batch_size=BATCH_SIZE, 
    shuffle=SHUFFLE, 
    num_workers=NUM_WORERS, 
    persistent_workers= PERSISTENT_WORKERS
)
test_loader = DataLoader(
    dataset_test, 
    batch_size=BATCH_SIZE, 
    shuffle=SHUFFLE, 
    num_workers=NUM_WORERS, 
    persistent_workers= PERSISTENT_WORKERS
)

## Plot some example images

In [ ]:
examples = next(iter(train_loader))
images, labels = examples
grid = make_grid(images[:9], nrow=3)
plt.imshow(grid.permute(1, 2, 0))

## Load ResNet50 model

In [ ]:
#Load best resNet50 weights
model = resnet50(weights=ResNet50_Weights.DEFAULT)

#Change output layer to 2 classes
model.fc = torch.nn.Linear(
    in_features=2048,
    out_features=2,
    bias=True
)

#Freeze all layers except fc
for name, param in model.named_parameters():
    if "fc.weight" in name or "fc.bias" in name:
        param.requires_grad = True
    else:
        param.requires_grad = False

## Create Lightning module

In [ ]:
class Classificator(L.LightningModule):
    def __init__(self, rn50, class_labels):
        super().__init__()
        self.rn50 = rn50
        self.class_labels = class_labels
        self.train_confusion_matrix = BinaryConfusionMatrix()
        self.val_confusion_matrix = BinaryConfusionMatrix()
        self.test_confusion_matrix = BinaryConfusionMatrix()
        metrics = MetricCollection([
            BinaryPrecision(),
            BinaryRecall(),
            BinaryF1Score(),
            BinaryAccuracy()
        ])
        self.train_metrics = metrics.clone(prefix='train_')
        self.valid_metrics = metrics.clone(prefix='val_')
        self.test_metrics = metrics.clone(prefix="test_")


    def training_step(self, batch):
        images, labels = batch
        output = self.rn50(images)
        _, preds = torch.max(output, dim=1)
        loss = F.cross_entropy(output, labels)
        self.log("Traning loss", loss, prog_bar=True)
        out_metric = self.train_metrics(preds, labels)
        self.log_dict(out_metric)
        self.train_confusion_matrix.update(preds, labels)
        return loss
    
    def on_train_epoch_end(self):
        cm = self.train_confusion_matrix.compute()
        image = self.transform_confusion_matrix(cm)
        self.logger.experiment.add_image("Confusion matrix train results", image)
        self.train_confusion_matrix.reset() 

    def validation_step(self, batch):
        images, labels = batch
        output = self.rn50(images)
        loss = F.cross_entropy(output, labels)
        _, preds = torch.max(output, dim=1)
        self.val_confusion_matrix.update(preds, labels)
        self.log("Validation loss", loss, prog_bar=True)
        self.valid_metrics.update(preds, labels)
        return loss
    
    def on_validation_epoch_end(self):
        output = self.valid_metrics.compute()
        self.log_dict(output)
        self.valid_metrics.reset()
        cm = self.val_confusion_matrix.compute()
        image = self.transform_confusion_matrix(cm)
        self.logger.experiment.add_image("Confusion matrix validation results", image)
        self.val_confusion_matrix.reset()
    
    def test_step(self, batch):
        images, labels = batch
        output = self.rn50(images)
        loss = F.cross_entropy(output, labels)
        _, preds = torch.max(output, dim=1)
        self.test_metrics.update(preds, labels)
        self.test_confusion_matrix.update(preds, labels)
        self.log("Test loss", loss, prog_bar=True)
        self.log_dict(self.test_metrics.compute())

    def on_test_end(self):
        cm = self.test_confusion_matrix.compute()
        image = self.transform_confusion_matrix(cm)
        self.logger.experiment.add_image("Confusion matrix test results", image)

    # Takes a tensor and plot confusion matrix from it and then return as tensor
    def transform_confusion_matrix(self, cm):
        cm = cm.cpu().numpy()
        fig = plt.figure()
        sns.heatmap(cm, annot=True, xticklabels=CLASS_LABELS, yticklabels=CLASS_LABELS, fmt="g", cbar=False)
        buf = io.BytesIO()
        fig.savefig(buf, format='png')
        buf.seek(0)
        image = PIL.Image.open(buf)
        image = ToTensor()(image)
        plt.close()
        return image


    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer

## Run training and validation loops

In [ ]:
classifier = Classificator(model, CLASS_LABELS)
early_stop_callback = EarlyStopping(monitor="Validation loss", min_delta=1e-6, patience=3)
checkpoint = L.pytorch.callbacks.ModelCheckpoint(dirpath="pneumonia_model/ResNet/")
callbacks = [early_stop_callback, checkpoint]

trainer = L.Trainer(
    accelerator="auto",
    devices=1,
    max_epochs=EPOCHS,
    callbacks=callbacks
)
trainer.fit(
    model = classifier,
    train_dataloaders = train_loader, 
    val_dataloaders = val_loader
)

## Test model
Only implemented and tested but should not use until final model is decided

In [ ]:
#trainer.test(model = classifier, dataloaders=test_loader)

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir=lightning_logs/